In [ ]:
import pandas as pd


In [ ]:
data = pd.read_csv('historico_de_vendas.csv')

# ETL - Histórico de Vendas

Este notebook processa o histórico de vendas e importa TODOS os dados para a tabela **OrderHistory**.

## Estratégia:
1. Carregar arquivo Excel com histórico de vendas
2. Processar e validar os dados
3. Gerar hash_cliente para cada id_unico_cliente
4. Geolocalizador CEP para obter coordenadas (opcional - somente clientes novos)
5. Inserir TODOS os registros na tabela **order_history**
6. Popular tabelas relacionadas: ClientName, LatLong, Products, Routs

## Tabelas Afetadas:
- **order_history**: TODOS os 33 campos do Excel + metadados
- **client_name**: Clientes únicos extraídos do histórico
- **latlong**: Coordenadas geográficas dos clientes
- **products**: Produtos únicos
- **routs**: Rotas geradas a partir de pedidos entregues

In [ ]:
# Imports necessários
import pandas as pd
from datetime import datetime
import sys
import os

# Adiciona o diretório raiz ao path
sys.path.append(os.path.abspath('../..'))

# Importa função centralizada de hash
from base.utils import generate_client_hash

print("✅ Bibliotecas carregadas com sucesso")
print("✅ Função de hash centralizada importada: generate_client_hash()")

In [ ]:
# Configuração do banco de dados
from base.models import db, OrderHistory, ClientName, LatLong, Products, Routs
from app import create_app

# Cria app Flask para acesso ao banco
app = create_app()
app.app_context().push()

print("✅ Conexão com banco de dados estabelecida")
print(f"📊 Tabela principal: OrderHistory (order_history_data)")
print(f"📊 Tabelas relacionadas: ClientName, LatLong, Products, Routs")

In [ ]:
# Carregar dados do histórico de vendas
df = pd.read_excel('historico_vendas_DF.xlsx')

print(f"📊 Total de registros carregados: {len(df):,}")
print(f"\n📋 Colunas disponíveis:")
for i, col in enumerate(df.columns, 1):
    print(f"   {i:2d}. {col}")
    
df.head()

## Passo 1: Preparar Dados para Inserção

Gerar hash dos clientes e preparar os campos para a tabela OrderHistory

In [ ]:
# ID do usuário que está importando (altere conforme necessário)
USER_ID = 1

# Gera hash para cada registro usando função centralizada
# IMPORTANTE: Esta função é a MESMA usada no ETL de clientes
# Garante que clientes com mesmo id_unico_cliente tenham o mesmo hash
df['hash_cliente'] = df['id_unico_cliente'].apply(generate_client_hash)

# Gera product_code baseado no id_produto
df['product_code'] = 'PROD_' + df['id_produto'].astype(str)

print(f"✅ Total de registros a serem importados: {len(df):,}")
print(f"✅ Clientes únicos: {df['id_unico_cliente'].nunique():,}")
print(f"✅ Produtos únicos: {df['id_produto'].nunique():,}")
print(f"✅ Pedidos únicos: {df['id_pedido'].nunique():,}")

print(f"\n🔐 Validação de Hash:")
print(f"   Algoritmo: MD5")
print(f"   Tamanho: 32 caracteres hexadecimais")
print(f"   Exemplo: {df['hash_cliente'].iloc[0]}")

# Mostra um exemplo do que será inserido
print(f"\n📋 Exemplo de registro:")
exemplo = df.iloc[0]
for col in ['id_pedido', 'id_unico_cliente', 'hash_cliente', 'cidade_cliente', 'estado_cliente']:
    if col in df.columns:
        print(f"   {col}: {exemplo[col]}")

In [ ]:
# Converter colunas de data para datetime
colunas_data = [
    'data_compra', 'data_aprovacao', 'data_envio_transportadora',
    'data_entrega_cliente', 'data_estimada_entrega', 'data_limite_envio',
    'data_criacao_avaliacao', 'data_resposta_avaliacao'
]

for col in colunas_data:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')

print("✅ Datas convertidas para datetime")
print(f"\n? Exemplo de datas:")
for col in colunas_data:
    if col in df.columns:
        print(f"   {col}: {df[col].iloc[0]}")

## Passo 2: Inserir TODOS os Registros na Tabela OrderHistory

Inserção completa do histórico de vendas

In [ ]:
# Inserir todos os registros na tabela OrderHistory
import time
from tqdm import tqdm

print("🔄 Iniciando inserção em OrderHistory...")
print(f"📊 Total de registros: {len(df):,}")
print(f"⏱️  Tempo estimado: ~{len(df) / 1000:.1f} minutos (1000 registros/min)")

registros_inseridos = 0
registros_erro = 0
batch_size = 100  # Inserir em lotes de 100

start_time = time.time()

for idx in range(0, len(df), batch_size):
    batch = df.iloc[idx:idx+batch_size]
    
    for _, row in batch.iterrows():
        try:
            novo_registro = OrderHistory(
                id_pedido=str(row['id_pedido']),
                id_item_pedido=int(row['id_item_pedido']) if pd.notna(row['id_item_pedido']) else None,
                id_cliente=str(row['id_cliente']) if pd.notna(row['id_cliente']) else None,
                id_unico_cliente=str(row['id_unico_cliente']),
                hash_cliente=str(row['hash_cliente']),
                id_produto=str(row['id_produto']) if pd.notna(row['id_produto']) else None,
                product_code=str(row['product_code']),
                
                # Datas
                data_compra=row['data_compra'] if pd.notna(row['data_compra']) else None,
                data_aprovacao=row['data_aprovacao'] if pd.notna(row['data_aprovacao']) else None,
                data_envio_transportadora=row['data_envio_transportadora'] if pd.notna(row['data_envio_transportadora']) else None,
                data_entrega_cliente=row['data_entrega_cliente'] if pd.notna(row['data_entrega_cliente']) else None,
                data_estimada_entrega=row['data_estimada_entrega'] if pd.notna(row['data_estimada_entrega']) else None,
                data_limite_envio=row['data_limite_envio'] if pd.notna(row['data_limite_envio']) else None,
                
                # Status e métricas
                status_pedido=str(row['status_pedido']) if pd.notna(row['status_pedido']) else None,
                tempo_entrega_dias=int(row['tempo_entrega_dias']) if pd.notna(row['tempo_entrega_dias']) else None,
                atraso_entrega_dias=int(row['atraso_entrega_dias']) if pd.notna(row['atraso_entrega_dias']) else None,
                
                # Campos calculados
                ano_compra=int(row['ano_compra']) if pd.notna(row['ano_compra']) else None,
                mes_compra=int(row['mes_compra']) if pd.notna(row['mes_compra']) else None,
                ano_mes_compra=str(row['ano_mes_compra']) if pd.notna(row['ano_mes_compra']) else None,
                dia_semana_compra=int(row['dia_semana_compra']) if pd.notna(row['dia_semana_compra']) else None,
                
                # Valores
                preco=float(row['preco']) if pd.notna(row['preco']) else None,
                valor_frete=float(row['valor_frete']) if pd.notna(row['valor_frete']) else None,
                valor_total_item=float(row['valor_total_item']) if pd.notna(row['valor_total_item']) else None,
                valor_total_pagamento=float(row['valor_total_pagamento']) if pd.notna(row['valor_total_pagamento']) else None,
                
                # Pagamento
                num_pagamentos=int(row['num_pagamentos']) if pd.notna(row['num_pagamentos']) else None,
                tipos_pagamento=str(row['tipos_pagamento']) if pd.notna(row['tipos_pagamento']) else None,
                max_parcelas=int(row['max_parcelas']) if pd.notna(row['max_parcelas']) else None,
                
                # Localização
                cidade_cliente=str(row['cidade_cliente']) if pd.notna(row['cidade_cliente']) else None,
                estado_cliente=str(row['estado_cliente']) if pd.notna(row['estado_cliente']) else None,
                cep_cliente=str(row['cep_cliente']) if pd.notna(row['cep_cliente']) else None,
                
                # Avaliação
                nota_avaliacao=int(row['nota_avaliacao']) if pd.notna(row['nota_avaliacao']) else None,
                titulo_comentario=str(row['titulo_comentario']) if pd.notna(row['titulo_comentario']) else None,
                mensagem_comentario=str(row['mensagem_comentario']) if pd.notna(row['mensagem_comentario']) else None,
                data_criacao_avaliacao=row['data_criacao_avaliacao'] if pd.notna(row['data_criacao_avaliacao']) else None,
                data_resposta_avaliacao=row['data_resposta_avaliacao'] if pd.notna(row['data_resposta_avaliacao']) else None,
                
                # Metadados
                user_id=USER_ID
            )
            
            db.session.add(novo_registro)
            registros_inseridos += 1
            
        except Exception as e:
            registros_erro += 1
            if registros_erro <= 5:  # Mostra apenas os primeiros 5 erros
                print(f"❌ Erro no registro {idx}: {str(e)}")
    
    # Commit a cada batch
    try:
        db.session.commit()
        if (idx + batch_size) % 1000 == 0:
            elapsed = time.time() - start_time
            registros_por_segundo = registros_inseridos / elapsed
            print(f"   ✅ Inseridos: {registros_inseridos:,} | Erros: {registros_erro} | Velocidade: {registros_por_segundo:.0f} reg/s")
    except Exception as e:
        db.session.rollback()
        print(f"❌ Erro no commit do batch {idx}: {str(e)}")

elapsed_time = time.time() - start_time

print("\n" + "="*80)
print(f"✅ INSERÇÃO CONCLUÍDA!")
print(f"   Total inserido: {registros_inseridos:,} registros")
print(f"   Total com erro: {registros_erro} registros")
print(f"   Tempo total: {elapsed_time:.1f} segundos ({elapsed_time/60:.1f} minutos)")
print(f"   Velocidade média: {registros_inseridos/elapsed_time:.0f} registros/segundo")
print("="*80)

## Passo 3: Verificar Clientes Existentes

Identificar quais clientes do histórico ainda não estão cadastrados na tabela ClientName

In [ ]:
# Extrair clientes únicos do histórico
clientes_unicos = df.groupby('hash_cliente').agg({
    'id_unico_cliente': 'first',
    'cidade_cliente': 'first',
    'estado_cliente': 'first',
    'cep_cliente': 'first'
}).reset_index()

print(f"📊 Total de clientes únicos no histórico: {len(clientes_unicos):,}")

# Verificar quais clientes já existem no banco
clientes_existentes = []
clientes_novos = []

for _, cliente in clientes_unicos.iterrows():
    existe = ClientName.query.filter_by(hash_client=cliente['hash_cliente']).first()
    
    if existe:
        clientes_existentes.append(cliente['hash_cliente'])
    else:
        clientes_novos.append({
            'hash_cliente': cliente['hash_cliente'],
            'id_unico_cliente': cliente['id_unico_cliente'],
            'cidade': cliente['cidade_cliente'],
            'estado': cliente['estado_cliente'],
            'cep': cliente['cep_cliente']
        })

print(f"\n✅ Clientes já cadastrados: {len(clientes_existentes):,}")
print(f"⚠️  Clientes NOVOS (não cadastrados): {len(clientes_novos):,}")

if len(clientes_novos) > 0:
    print(f"\n📋 Lista de clientes novos (primeiros 10):")
    df_novos = pd.DataFrame(clientes_novos)
    print(df_novos.head(10).to_string(index=False))
    
    # Salvar lista completa de clientes novos para análise posterior
    global clientes_novos_df
    clientes_novos_df = df_novos
else:
    print(f"\n✅ Todos os clientes do histórico já estão cadastrados!")
    clientes_novos_df = pd.DataFrame()

## Passo 4: Popular Tabela Products

Extrair produtos únicos e inserir na tabela Products

In [ ]:
# Extrair produtos únicos com preço médio
produtos_unicos = df.groupby(['id_produto', 'product_code']).agg({
    'preco': 'mean'
}).reset_index()

print(f"📊 Total de produtos únicos: {len(produtos_unicos):,}")

# Inserir produtos no banco
produtos_inseridos = 0
produtos_existentes = 0

for _, produto in produtos_unicos.iterrows():
    existe = Products.query.filter_by(product_code=produto['product_code']).first()
    
    if not existe:
        novo_produto = Products(
            product_name=f"Produto {produto['id_produto'][:15]}",
            product_code=produto['product_code'],
            product_type='Geral',
            product_value=float(produto['preco']) if pd.notna(produto['preco']) else 0.0
        )
        db.session.add(novo_produto)
        produtos_inseridos += 1
    else:
        produtos_existentes += 1

db.session.commit()

print(f"✅ Produtos novos inseridos: {produtos_inseridos:,}")
print(f"ℹ️  Produtos já existentes: {produtos_existentes:,}")

## Passo 5: Verificação e Estatísticas

Consultar os dados inseridos e gerar estatísticas do histórico

In [ ]:
# Consultar estatísticas da tabela OrderHistory
print("="*80)
print("📊 ESTATÍSTICAS DO HISTÓRICO DE VENDAS")
print("="*80)

# Total de registros
total_registros = OrderHistory.query.filter_by(user_id=USER_ID).count()
print(f"\n✅ Total de registros importados: {total_registros:,}")

# Por status de pedido
print(f"\n📦 Distribuição por Status:")
status_counts = db.session.query(
    OrderHistory.status_pedido,
    db.func.count(OrderHistory.id)
).filter_by(user_id=USER_ID).group_by(OrderHistory.status_pedido).all()

for status, count in status_counts:
    percentual = (count / total_registros) * 100
    print(f"   {status}: {count:,} ({percentual:.1f}%)")

# Por ano
print(f"\n📅 Distribuição por Ano:")
ano_counts = db.session.query(
    OrderHistory.ano_compra,
    db.func.count(OrderHistory.id)
).filter_by(user_id=USER_ID).group_by(OrderHistory.ano_compra).order_by(OrderHistory.ano_compra).all()

for ano, count in ano_counts:
    if ano:
        print(f"   {ano}: {count:,} pedidos")

# Valores financeiros
print(f"\n💰 Análise Financeira:")
valor_total = db.session.query(
    db.func.sum(OrderHistory.valor_total_pagamento)
).filter_by(user_id=USER_ID).scalar()

valor_medio = db.session.query(
    db.func.avg(OrderHistory.valor_total_pagamento)
).filter_by(user_id=USER_ID).scalar()

print(f"   Valor total de vendas: R$ {valor_total:,.2f}" if valor_total else "   Valor total: N/A")
print(f"   Ticket médio: R$ {valor_medio:,.2f}" if valor_medio else "   Ticket médio: N/A")

# Tempo de entrega
print(f"\n🚚 Análise de Entregas:")
tempo_medio = db.session.query(
    db.func.avg(OrderHistory.tempo_entrega_dias)
).filter_by(user_id=USER_ID).scalar()

atraso_medio = db.session.query(
    db.func.avg(OrderHistory.atraso_entrega_dias)
).filter_by(user_id=USER_ID).scalar()

print(f"   Tempo médio de entrega: {tempo_medio:.1f} dias" if tempo_medio else "   Tempo médio: N/A")
print(f"   Atraso médio: {atraso_medio:.1f} dias" if atraso_medio else "   Atraso médio: N/A")

# Avaliações
print(f"\n⭐ Análise de Avaliações:")
nota_media = db.session.query(
    db.func.avg(OrderHistory.nota_avaliacao)
).filter_by(user_id=USER_ID).scalar()

total_avaliacoes = OrderHistory.query.filter_by(user_id=USER_ID).filter(
    OrderHistory.nota_avaliacao.isnot(None)
).count()

print(f"   Nota média: {nota_media:.2f} / 5.00" if nota_media else "   Nota média: N/A")
print(f"   Total de avaliações: {total_avaliacoes:,} ({(total_avaliacoes/total_registros)*100:.1f}%)")

## Passo 6: Resumo Final e Próximos Passos

In [ ]:
# Resumo de todas as tabelas populadas
print("\n" + "="*80)
print("📊 RESUMO FINAL - ETL HISTÓRICO DE VENDAS")
print("="*80)

print(f"\n✅ OrderHistory (Histórico Completo):")
total_order_history = OrderHistory.query.filter_by(user_id=USER_ID).count()
print(f"   Total de registros importados: {total_order_history:,}")

print(f"\n📊 ClientName (Clientes no Sistema):")
total_clientes_sistema = ClientName.query.filter_by(user_id=USER_ID).count()
print(f"   Total de clientes cadastrados: {total_clientes_sistema:,}")
print(f"   ✅ Clientes do histórico já cadastrados: {len(clientes_existentes):,}")
if len(clientes_novos_df) > 0:
    print(f"   ⚠️  Clientes novos detectados: {len(clientes_novos_df):,}")
    print(f"   👉 Revise e decida sobre inserção nas próximas células!")
else:
    print(f"   ✅ Todos os clientes do histórico já estão cadastrados!")

print(f"\n✅ Products (Produtos):")
total_produtos = Products.query.count()
print(f"   Total de produtos cadastrados: {total_produtos:,}")

print(f"\n📍 LatLong (Coordenadas Geográficas):")
total_coords = LatLong.query.filter_by(id_user=USER_ID).count()
print(f"   Total de coordenadas: {total_coords:,}")
print(f"   ℹ️  Use o ETL específico de geolocalização para popular esta tabela")

print("\n" + "="*80)
print("💡 PRÓXIMOS PASSOS:")
print("="*80)
print("1. ⚠️  Revise os CLIENTES NOVOS detectados (células abaixo)")
print("2. 🗺️  Execute ETL de geolocalização para popular LatLong com novos clientes")
print("3. 📊 Use OrderHistory para análises e relatórios completos")
print("4. 🧠 Gere features para ML usando dados agregados de OrderHistory")
print("5. 📈 Crie dashboard de visualização do histórico importado")
print("\n" + "="*80)
print("✅ IMPORTAÇÃO DO HISTÓRICO CONCLUÍDA!")
print("="*80)

## 🔴 ATENÇÃO: Clientes Novos Detectados

Existem clientes no histórico que NÃO estão cadastrados na tabela ClientName.
Execute a célula abaixo para revisar e decidir se deseja cadastrá-los.

In [ ]:
# Verificar se existem clientes novos para inserir
if len(clientes_novos_df) > 0:
    print("="*80)
    print("⚠️  CLIENTES NOVOS DETECTADOS")
    print("="*80)
    print(f"\nTotal de clientes novos: {len(clientes_novos_df):,}")
    print("\nEstes clientes aparecem no histórico mas NÃO estão cadastrados no sistema.")
    print("\n📋 Visualização dos clientes novos:")
    print(clientes_novos_df.to_string(index=True, max_rows=20))
    
    # Exportar para análise
    arquivo_clientes_novos = 'clientes_novos_detectados.xlsx'
    clientes_novos_df.to_excel(arquivo_clientes_novos, index=False)
    print(f"\n✅ Lista completa exportada para: {arquivo_clientes_novos}")
    
    print("\n" + "="*80)
    print("💡 OPÇÕES:")
    print("="*80)
    print("1. Revise o arquivo Excel gerado")
    print("2. Se desejar cadastrar estes clientes, execute a PRÓXIMA célula")
    print("3. Para cadastro seletivo, edite o DataFrame 'clientes_novos_df' antes de executar")
    print("="*80)
else:
    print("✅ Nenhum cliente novo detectado. Todos já estão cadastrados!")

## 🔧 OPCIONAL: Inserir Clientes Novos no Sistema

**Execute esta célula SOMENTE se você revisou os clientes novos e deseja cadastrá-los**

In [ ]:
# Função para inserir clientes novos após confirmação do usuário
def inserir_clientes_novos(confirmar=False):
    """
    Insere os clientes novos na tabela ClientName
    
    Parâmetros:
        confirmar (bool): True para confirmar a inserção, False para apenas simular
    """
    
    if len(clientes_novos_df) == 0:
        print("✅ Nenhum cliente novo para inserir!")
        return
    
    if not confirmar:
        print("⚠️  MODO SIMULAÇÃO")
        print("="*80)
        print(f"📊 {len(clientes_novos_df):,} clientes seriam inseridos")
        print("\nPara confirmar a inserção, execute:")
        print("    inserir_clientes_novos(confirmar=True)")
        print("="*80)
        return
    
    # Confirmação real de inserção
    print("🔄 Iniciando inserção de clientes novos...")
    print("="*80)
    
    clientes_inseridos = 0
    clientes_erro = 0
    
    for idx, cliente in clientes_novos_df.iterrows():
        try:
            # Verifica novamente se não existe (segurança)
            existe = ClientName.query.filter_by(hash_client=cliente['hash_cliente']).first()
            
            if not existe:
                novo_cliente = ClientName(
                    name_client=f"Cliente_{cliente['hash_cliente'][:12]}",
                    hash_client=cliente['hash_cliente'],
                    user_id=USER_ID,
                    cidade=cliente['cidade'],
                    estado=cliente['estado']
                )
                db.session.add(novo_cliente)
                clientes_inseridos += 1
                
                if (clientes_inseridos) % 100 == 0:
                    db.session.commit()
                    print(f"   ✅ Inseridos: {clientes_inseridos:,}")
            else:
                print(f"   ⚠️  Cliente {cliente['hash_cliente'][:8]}... já existe, pulando")
                
        except Exception as e:
            clientes_erro += 1
            print(f"   ❌ Erro ao inserir cliente {idx}: {str(e)}")
            db.session.rollback()
    
    # Commit final
    try:
        db.session.commit()
        print("\n" + "="*80)
        print(f"✅ INSERÇÃO CONCLUÍDA!")
        print(f"   Clientes inseridos: {clientes_inseridos:,}")
        print(f"   Erros: {clientes_erro}")
        print("="*80)
        
        # Atualizar contagem
        total_clientes_atual = ClientName.query.filter_by(user_id=USER_ID).count()
        print(f"\n📊 Total de clientes no sistema: {total_clientes_atual:,}")
        
    except Exception as e:
        db.session.rollback()
        print(f"\n❌ Erro no commit final: {str(e)}")

# Chama a função em modo simulação primeiro
print("💡 Executando em MODO SIMULAÇÃO...")
inserir_clientes_novos(confirmar=False)

### ✅ Para CONFIRMAR a inserção, execute a linha abaixo:

```python
inserir_clientes_novos(confirmar=True)
```

In [ ]:
# Descomente a linha abaixo para CONFIRMAR a inserção dos clientes novos
# inserir_clientes_novos(confirmar=True)